In [5]:
import glob
import os
import librosa
import numpy as np
!pip install pretty_midi
import pretty_midi

     |████████████████████████████████| 5.6 MB 4.4 MB/s 
     |████████████████████████████████| 51 kB 5.5 MB/s 
  Created wheel for pretty-midi: filename=pretty_midi-0.2.9-py3-none-any.whl size=5591955 sha256=77c61372c732ae21981c9de5eeb7c4ce39a58d38fd5300523c8d717d9a8e7428
  Stored in directory: /root/.cache/pip/wheels/ad/74/7c/a06473ca8dcb63efb98c1e67667ce39d52100f837835ea18fa
Successfully built pretty-midi


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True) 

Mounted at /content/drive


**Please update the start path and destination path**

In [3]:
start ='/content/drive/MyDrive/MUS' # Divide all 9 directories of the MAPS Dataset into train/test/val and provide its path (format eg. test/(ENS..)/MUS/)
dest = '/content/drive/MyDrive/finale' # Destination Path for storing the .npy files

In [6]:
RangeMIDInotes=[21,108]
n_fft = 2048
sr=16000.
n_mels = 252
bins_per_octave=36
n_octave=7
val_rate=1./7
pretty_midi.pretty_midi.MAX_TICK = 1e10
n_bins= n_octave * bins_per_octave
hop_length = 512
win_width = 32
kernel_size=7
overlap=True

In [7]:
def midi2mat(midi_path_train, length, mel_len, sr, RangeMIDInotes=RangeMIDInotes):
    midi_data = pretty_midi.PrettyMIDI(midi_path_train)
    pianoRoll = midi_data.instruments[0].get_piano_roll(fs=mel_len * sr/length)
    Ground_truth_mat = (pianoRoll[RangeMIDInotes[0]:RangeMIDInotes[1] + 1, :mel_len] > 0)
    return Ground_truth_mat

In [ ]:
fil = [direc for direc in os.listdir(start)]
for direc in fil:
    j=0
    k=0
    startpath= os.path.join(start,direc)
    destpath = os.path.join(dest,direc)
    if not os.path.exists(destpath):
        os.makedirs(destpath)
    print(direc)
    files = [f for f in os.listdir(startpath)]
    for f in files:
        fpath=startpath
        f1=f
        # # print(fpath)
        # subfiles = [f1 for f1 in os.listdir(fpath)]    
        if 1:
            ffile=os.path.join(fpath,f1)
            file_name,file_extensions=os.path.splitext(f1)
            if file_extensions == '.txt':
                continue
            if file_extensions==".mid":
                ffile=os.path.join(fpath,file_name+'.wav')
            x,sr = librosa.load(ffile,sr=sr)
            
            mel_spectrogram = librosa.feature.melspectrogram(x, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels)
            mel = np.transpose(np.abs(mel_spectrogram))
            mel = np.transpose(librosa.power_to_db(np.abs(mel_spectrogram)))

            midi_file = os.path.join(fpath,f1)

            if file_extensions==".wav":
                midi_file = os.path.join(fpath,file_name+'.mid')

            Ground_truth_mat=midi2mat(midi_file, len(x), mel.shape[0], sr, RangeMIDInotes=RangeMIDInotes)
            midi_train = np.transpose(Ground_truth_mat)
            
            #midi length < MEL length, cut MEL
            if midi_train.shape[0]<mel.shape[0]:
                mel=mel[:midi_train.shape[0],:]
        
            if file_extensions == ".wav" :
                ofolder = 'wav'
                subname = 'MEL'
                no=j
            elif file_extensions == ".mid" :
                ofolder = 'mid'
                subname = 'label'
                no=k

            opath = os.path.join(destpath,f,ofolder,file_name)+subname+'.npy'
            temp_path = os.path.join(destpath,f,ofolder)

            if not os.path.exists(temp_path):
                os.makedirs(temp_path)

            if file_extensions == ".wav":
                np.save(opath,mel)
            elif file_extensions == ".mid":
                np.save(opath,midi_train)

            #print('Preprocessed',f1,"no",no)   
            
            matrix = np.array(np.load(opath))
            l=matrix.shape[0]
            cut_matrix=[]
            nb_win=int(l/win_width)   #integer division=floor

            if not overlap:
                for i in range(nb_win):
                    cut_matrix.append(matrix[i*win_width:(i+1)*win_width,:])
            else:
                w=matrix.shape[1]
                matrix_1=np.concatenate([np.zeros([int(kernel_size/2),w]),matrix,np.zeros([int(kernel_size/2),w])],axis=0)  #padding
                cut_matrix = []
                for i in range(nb_win):
                    cut_matrix.append(matrix_1[i * win_width:(i + 1) * win_width+kernel_size-1,:])    
            cut_matrix = np.asarray(cut_matrix)
            os.remove(opath)
            #print("Removed",f1,"no:",no)

            if file_extensions == ".wav":
                if j == 0:
                    X = cut_matrix
                    #print(cut_matrix.shape)
                else:
                    X = np.concatenate((X,cut_matrix),axis=0)
                    #print(cut_matrix.shape)
                j=j+1
                
            elif file_extensions == ".mid":
                if k == 0:
                    Y = cut_matrix
                    #print(cut_matrix.shape)
                else:
                    Y = np.concatenate((Y,cut_matrix),axis=0)
                    #print(cut_matrix.shape)
                k=k+1
            print('Joined ',f1,"no ",no)
            #print('--------------')
            os.rmdir(temp_path)
        os.rmdir(os.path.join(destpath,f))

    X = np.expand_dims(X,axis=-2)
    Y = np.expand_dims(Y,axis=-2) 

    X_train = np.expand_dims(X_train,axis=-2)
    Y_train = np.expand_dims(Y_train,axis=-2)

    opath1= os.path.join(destpath,"X_final_MEL_")+direc+'.npy'
    opath2= os.path.join(destpath,"Y_final_MEL_")+direc+'.npy' 
    np.save(opath1,X)
    np.save(opath2,Y)
    # print('Saved X_train final')     
    # print('Saved Y_train final')
    # print('X_train_Shape -',X.shape)     
    # print('Y_train_Shape -',Y.shape)